# Fuzzy String Matching untuk Data Transaksi

## Deskripsi
Notebook ini mendemonstrasikan penggunaan **fuzzy string matching** untuk mencocokkan data transaksi dengan profil pelanggan dan master produk. Fuzzy matching berguna ketika data tidak persis sama tetapi memiliki kemiripan (typo, variasi penulisan, dll).

## Tujuan
1. **Mencocokkan nama pelanggan** di transaksi dengan nama lengkap di profil pelanggan
2. **Mencocokkan alamat pengiriman** dengan alamat domisili untuk validasi
3. **Mencocokkan nama barang** di transaksi dengan nama produk resmi di master produk
4. **Menghasilkan tabel final** yang sudah diperkaya dengan informasi lengkap

## Library yang Digunakan
- `pandas` - untuk manipulasi data
- `thefuzz` - untuk fuzzy string matching (pengganti FuzzyWuzzy)
- `python-Levenshtein` - untuk performa fuzzy matching yang lebih cepat

## Struktur Data
Notebook ini menggunakan 3 sheet dari file Excel:
- **profil_pelanggan**: Data profil lengkap pelanggan
- **transaksi_pembelian**: Data transaksi yang akan diperkaya
- **master_produk**: Data produk resmi sebagai referensi

## Algoritma Fuzzy Matching
- **token_sort_ratio**: Untuk mencocokkan nama (mengabaikan urutan kata)
- **token_set_ratio**: Untuk mencocokkan produk dan alamat (mengabaikan duplikasi kata)

## Threshold Scoring
- **Skor Nama**: ≥ 70 (dari skala 0-100)
- **Skor Alamat**: ≥ 75 (dari skala 0-100)  
- **Skor Produk**: ≥ 75 (dari skala 0-100)

## Cara Menjalankan

### Langkah 1: Install Dependencies
Jalankan cell pertama untuk menginstall library yang diperlukan:
```python
%pip install thefuzz python-Levenshtein
```

### Langkah 2: Import dan Setup
Jalankan cell berikutnya yang berisi import library dan setup path file Excel.

### Langkah 3: Eksekusi Fuzzy Matching
Kode akan secara otomatis:
1. **Membaca data** dari 3 sheet Excel
2. **Tahap 1**: Mencocokkan transaksi dengan profil pelanggan
3. **Tahap 2**: Mencocokkan hasil dengan master produk  
4. **Menampilkan hasil** final yang sudah diperkaya

### Output yang Dihasilkan
- Analisis skor sebelum filtering
- Tabel final dengan kolom:
  - Data transaksi original
  - Data profil pelanggan yang cocok
  - Data produk resmi yang cocok
  - Skor kemiripan untuk setiap matching

---

## Kode Implementation

In [ ]:
import pandas as pd
from thefuzz import process, fuzz
import os

# ===================================================================
# MEMBACA DATA DARI FILE EXCEL ANDA
# ===================================================================
file_path = r"C:\Users\user\OneDrive - untirta.ac.id\UNTIRTA\Bahan Ajar\Pengantar Data Sains\25-26\Contoh Data Penjualan Toko.xlsx"

if not os.path.exists(file_path):
    print(f"Error: File tidak ditemukan di path: {file_path}")
    exit()

df_profil = pd.read_excel(file_path, sheet_name='profil_pelanggan')
df_transaksi = pd.read_excel(file_path, sheet_name='transaksi_pembelian')
df_produk = pd.read_excel(file_path, sheet_name='master_produk')

print("--- Data Berhasil Dimuat dari Excel ---")

# ===================================================================
# LOGIKA FUZZY MATCHING
# ===================================================================

def find_best_match(name, choices_list, scorer=fuzz.token_set_ratio):
    best_match = process.extractOne(name, choices_list, scorer=scorer)
    if best_match:
        return best_match[0], best_match[1]
    return None, None

# TAHAP 1
print("\n" + "="*50 + "\n")
print("TAHAP 1: Mencocokkan Transaksi dengan Profil (Nama & Alamat)...")

choices_nama = df_profil['nama_lengkap'].tolist()
match_results_nama = df_transaksi['nama_pelanggan'].apply(lambda x: find_best_match(x, choices_nama))
df_transaksi[['profil_cocok', 'skor_nama']] = pd.DataFrame(match_results_nama.tolist(), index=df_transaksi.index)

df_transaksi_intermediate = pd.merge(df_transaksi, df_profil, left_on='profil_cocok', right_on='nama_lengkap', how='left')

def calculate_address_score(row):
    if pd.notna(row['alamat_pengiriman']) and pd.notna(row['alamat_domisili']):
        return fuzz.token_set_ratio(row['alamat_pengiriman'], row['alamat_domisili'])
    return 0
df_transaksi_intermediate['skor_alamat'] = df_transaksi_intermediate.apply(calculate_address_score, axis=1)


print("\n--- Analisis Skor (Sebelum Filter) ---")
print(df_transaksi_intermediate[['id_transaksi', 'nama_pelanggan', 'profil_cocok', 'skor_nama', 'skor_alamat']].to_string())


# Filter berdasarkan skor nama DAN skor alamat
# =================================================================
SKOR_NAMA_THRESHOLD = 70 
# =================================================================
SKOR_ALAMAT_THRESHOLD = 75

df_linked_profil = df_transaksi_intermediate[
    (df_transaksi_intermediate['skor_nama'] >= SKOR_NAMA_THRESHOLD) &
    (df_transaksi_intermediate['skor_alamat'] >= SKOR_ALAMAT_THRESHOLD)
].copy() 


# TAHAP 2
print("\nTAHAP 2: Mencocokkan Transaksi dengan Daftar Produk...")

choices_produk = df_produk['nama_produk_resmi'].tolist()
match_results_produk = df_linked_profil['nama_barang'].apply(lambda x: find_best_match(x, choices_produk, scorer=fuzz.token_set_ratio))
df_linked_profil.loc[:, 'produk_cocok'] = match_results_produk.apply(lambda x: x[0] if x else None)
df_linked_profil.loc[:, 'skor_produk'] = match_results_produk.apply(lambda x: x[1] if x else 0)

df_final = pd.merge(df_linked_profil, df_produk, left_on='produk_cocok', right_on='nama_produk_resmi', how='left')

SKOR_PRODUK_THRESHOLD = 75
df_final = df_final[df_final['skor_produk'] >= SKOR_PRODUK_THRESHOLD]

# TAHAP AKHIR
print("\n" + "="*50 + "\n")
print("HASIL AKHIR: Tabel Transaksi yang Diperkaya dan Rapi (HASIL FINAL)")
final_columns = [
    'id_transaksi', 'id_profil', 'nama_pelanggan', 'nama_lengkap',
    'id_produk', 'nama_barang', 'nama_produk_resmi',
    'jumlah', 'harga', 'skor_nama', 'skor_alamat', 'skor_produk'
]
final_columns_exist = [col for col in final_columns if col in df_final.columns]
print(df_final[final_columns_exist].to_string())

--- Data Berhasil Dimuat dari Excel ---


TAHAP 1: Mencocokkan Transaksi dengan Profil (Nama & Alamat)...

--- Analisis Skor (Sebelum Filter) ---
  id_transaksi   nama_pelanggan            profil_cocok  skor_nama  skor_alamat
0        TRX01       Ferdian B.  Ferdian Bangkit Wijaya         88           91
1        TRX02         Weksi B.           Weksi Budiaji         83           94
2        TRX03  Chintia Lestari    Cynthia Dewi Lestari         74           93
3        TRX04    Ahmad Zaelany           Ahmad Zaelani         92           81
4        TRX05    Weksi Budiaji           Weksi Budiaji        100           77

TAHAP 2: Mencocokkan Transaksi dengan Daftar Produk...


HASIL AKHIR: Tabel Transaksi yang Diperkaya dan Rapi (HASIL FINAL)
  id_transaksi id_profil   nama_pelanggan            nama_lengkap id_produk             nama_barang        nama_produk_resmi  jumlah  harga  skor_nama  skor_alamat  skor_produk
0        TRX01      P001       Ferdian B.  Ferdian Bangkit Wijaya    PRO